In [21]:
import numpy as np
import pandas as pd
from xml.etree import ElementTree as ET

### Warm-up exercise

In [226]:
membership = ET.parse('./data/data_wrangling_xml/data/membership.xml')

In [23]:
users = membership.find('users')

In [24]:
for user in membership.findall('users/user'):
    print(user.attrib['name'])

john
charles
peter


In [25]:
for group in membership.findall('groups/group'):
    print(group.attrib['name'])

users
administrators


In [27]:
for group in membership.findall('groups/group'):
    print('group:' , group.attrib['name'])
    print('users:')
    for node in group.getchildren():
        if node.tag == 'user':
            print('-', node.attrib['name'])

group: users
users:
- john
- charles
group: administrators
users:
- peter


In [29]:
#1. iterate nodes including the starting point
users = membership.find('users')
for node in users.getiterator():
    print(node.tag,node.attrib,node.text,node.tail)

users {} 
         

    
user {'name': 'john'} None 
        
user {'name': 'charles'} None 
        
user {'name': 'peter'} None 
    


In [31]:
#2. iterate only the children
users = membership.find('users')
for node in users.getchildren():
    print(node.tag,node.attrib,node.text,node.tail)

user {'name': 'john'} None 
        
user {'name': 'charles'} None 
        
user {'name': 'peter'} None 
    


# XML exercise
#### Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

## 1. 10 countries with the lowest infant mortality rates
## 2. 10 cities with the largest population
## 3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
## 4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [252]:
mondial = ET.parse( './data/data_wrangling_xml/data/mondial_database.xml' )
mondial_less = ET.parse( './data/data_wrangling_xml/data/mondial_database_less.xml' )

In [232]:
type(mondial)

xml.etree.ElementTree.ElementTree

In [233]:
countries = mondial.find('country')

In [234]:
for c in countries:
    for node in c.getiterator():
        if node.tag == 'name':
            print('-',node.text)
##not the expected behavior....

- Albania
- Tirana
- Tirane
- Shkodër
- Durrës
- Vlorë
- Elbasan
- Korçë


In [235]:
df1 = pd.DataFrame(columns=('country','infant_mortality'))
d = {'country':'','infant_mortality':''}
cou = 0
for c in mondial.findall('country'):
    for n in c.getchildren():
        if n.tag == 'name':
            #print(cou, n.text)
            d['country'] = n.text
        if n.tag == 'infant_mortality':
            #print(n.text)
            d['infant_mortality'] = n.text
            #print(cou, d)
            df1.loc[cou] = d
            cou += 1

In [236]:
df1.head()

,country,infant_mortality
0,Albania,13.19
1,Greece,4.78
2,Macedonia,7.9
3,Serbia,6.16
4,Andorra,3.69


# 1. 10 countries with the lowest infant mortality rates

In [237]:
df1.sort_values('infant_mortality', ascending=True).head(10)

,country,infant_mortality
36,Monaco,1.81
28,Romania,10.16
142,Fiji,10.2
63,Brunei,10.48
124,Grenada,10.5
221,Mauritius,10.59
116,Panama,10.7
227,Seychelles,10.77
94,United Arab Emirates,10.92
105,Barbados,10.93


In [238]:
df2 = pd.DataFrame(columns=('country', 'province', 'city', 'population'))
d = {'country':'', 'province':'', 'city':'', 'population':''}
cou = 0
for c in mondial.findall('country'):
    for n in c.getchildren():
        if n.tag == 'name':
            #print(n.text)
            d['country'] = n.text
        if n.tag == 'city':
            for nd in n.getchildren():
                if nd.tag == 'name':
                    #print(cou,'-',nd.text)
                    d['city'] = nd.text
                if nd.tag == 'population':
                    if nd.attrib['year'] == '2011':
                        d['population'] = int(nd.text)
                        d['province'] = 'none'
                df2.loc[cou] = d
            cou += 1
        if n.tag == 'province':
            d['province'] = n.getchildren()[0].text
            #print('province:', n.getchildren()[0].text)
            for c in n.findall('city'):
                for nd in c.getchildren():
                    if nd.tag == 'name':
                        #print(cou, '-',nd.text)
                        d['city'] = nd.text
                    if nd.tag == 'population':
                        if nd.attrib['year'] == '2011':
                            d['population'] = int(nd.text)
                            #print('pop:',nd.attrib['year'], nd.text)
                    df2.loc[cou] = d
                cou += 1

In [239]:
df2.head(20)

,country,province,city,population
0,Albania,none,Tirane,418495
1,Albania,none,Shkodër,77075
2,Albania,none,Durrës,113249
3,Albania,none,Vlorë,79513
4,Albania,none,Elbasan,78703
5,Albania,none,Korçë,51152
6,Greece,Anatolikis Makedonias kai Thrakis,Komotini,51152
7,Greece,Anatolikis Makedonias kai Thrakis,Kavala,58790
8,Greece,Attikis,Athina,664046
9,Greece,Attikis,Piräus,163688


In [261]:
len(df2)

3380

# 2. 10 cities with the largest population

In [240]:
df2.sort_values('population', ascending=False).head(10)
#hometown tops the list :D :D :D

,country,province,city,population
1527,India,Maharashtra,Mumbai,12442373
1582,India,Delhi,Delhi,11034555
1515,India,Karnataka,Bangalore,8443675
1517,India,Karnataka,Mysore,8443675
1516,India,Karnataka,Hubli,8443675
1000,United Kingdom,London,London,8250205
1382,Iran,Tehran,Tehran,8154051
1470,Bangladesh,Dhaka,Dhaka,7423137
1593,India,Uttarakhand,Dehra Dun,6731790
1594,Bhutan,Uttarakhand,Thimphu,6731790


# 3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)


# 4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [306]:
cou = 0
rivers = mondial.findall('river')
df4 = pd.DataFrame(columns=('name', 'country', 'length'))
d = {'name':'', 'country':'', 'length':''}
for r in rivers:
    try:
        t = r.attrib['country']
        n = r.getchildren()[0].text
        l = int(r.getchildren()[3].text)
    except:
        t = 'none'
        n = "none"
        l = 0
    #print(c, a.getchildren()[0].text, a.getchildren()[3].text)
    d['country'] = t
    d['name'] = n
    d['length'] = l
    df4.loc[cou] = d
    cou += 1

In [307]:
len(df4)

238

In [292]:
df4.head()

,name,country,length
0,Thjorsa,IS,230
1,none,none,0
2,Glomma,N,604
3,none,none,0
4,Goetaaelv,S,50180


## 4a. name and country of the longest river (top 10)

In [294]:
df4.sort_values('length', ascending=False).head(10)

,name,country,length
190,Nile,ET SUD,3254853
159,Mississippi,USA,2981076
114,Ob,R,2972497
119,Jenissej,R,2554482
123,Lena,R,2306772
137,Jangtse,CN,1722155
147,Nelson River,CDN,1093442
232,Vaal,RSA,973000
231,Oranje,LS NAM RSA,973000
146,Mackenzie River,CDN,886300


In [302]:
cou = 0
lakes = mondial.findall('lake')
df5 = pd.DataFrame(columns=('name', 'country', 'area'))
d = {'name':'', 'country':'', 'area':''}
for l in lakes:
    try:
        t = l.attrib['country']
        n = l.getchildren()[0].text
        a = int(l.getchildren()[3].text)
    except:
        t = 'none'
        n = "none"
        a = 0
    #print(t, n, a)
    d['country'] = t
    d['name'] = n
    d['area'] = a
    df5.loc[cou] = d
    cou += 1

In [303]:
len(df5)

141

In [308]:
df5.head(10)

,name,country,area
0,Inari,SF,1040
1,Oulujaervi,SF,928
2,Kallavesi,SF,472
3,Saimaa,SF,4370
4,Paeijaenne,SF,1118
5,Mjoesa-See,N,368
6,Storuman,S,173
7,Siljan,S,290
8,Maelaren,S,1140
9,Vaenern,S,5648


In [304]:
cou = 0
airports = mondial.findall('airport')
df3 = pd.DataFrame(columns=('country', 'city', 'airport', 'elevation'))
d = {'country':'', 'city':'', 'airport':'', 'elevation':''}
for a in airports:
    try:
        t = a.attrib['country']
        c = a.attrib['city']
        n = a.getchildren()[0].text
        e = int(a.getchildren()[3].text)
    except:
        t = 'none'
        c = "none"
        n = "none"
        e = 0
    #print(c, a.getchildren()[0].text, a.getchildren()[3].text)
    d['country'] = t
    d['city'] = c
    d['airport'] = n
    d['elevation'] = e
    df3.loc[cou] = d
    cou += 1

## 4b. name and country of the largest lake (top 10)

In [310]:
df5.sort_values('area', ascending=False).head(10)

,name,country,area
81,Lake Victoria,EAT EAK EAU,68870
108,Lake Michigan,USA,57800
43,Ozero Baikal,R,31492
99,Great Slave Lake,CDN,28568
101,Lake Winnipeg,CDN,24420
40,Ozero Ladoga,R,18400
55,Ozero Aral,UZB KAZ,17160
129,Lake Maracaibo,YV,13000
85,Lake Bangweulu,Z,10000
66,Lake Volta,GH,8502


In [284]:
df3.head(10)

,country,city,airport,elevation
0,AFG,cty-Afghanistan-2,Herat,977
1,AFG,cty-Afghanistan-Kabul,Kabul Intl,1792
2,AL,cty-Albania-Tirane,Tirana Rinas,38
3,DZ,cty-Algeria-14,Cheikh Larbi Tebessi,811
4,DZ,cty-Algeria-6,Batna Airport,822
5,DZ,cty-Algeria-11,Soummam,6
6,DZ,cty-Algeria-19,Tamanrasset,1377
7,DZ,cty-Algeria-17,Biskra,88
8,DZ,cty-Algeria-4,Mohamed Boudiaf Intl,691
9,DZ,cty-Algeria-7,Ain Arnat Airport,1024


In [305]:
len(df3)

1315

## 4c. Name and Country of the highest airport (top 10)

In [286]:
df3.sort_values('elevation', ascending=False).head(10)

,country,city,airport,elevation
80,BOL,cty-BOL-1,El Alto Intl,4063
219,CN,cty-China-324,Lhasa-Gonggar,4005
241,CN,cty-China-281,Yushu Batang,3963
813,PE,cty-Peru-15,Juliaca,3827
815,PE,cty-Peru-10,Teniente Alejandro Velasco Astete Intl,3311
82,BOL,cty-BOL-7,Juana Azurduy De Padilla,2905
334,EC,cty-Ecuador-Quito,Mariscal Sucre Intl,2813
805,PE,cty-Peru-18,Coronel Fap Alfredo Mendivil Duarte,2719
807,PE,cty-Peru-19,Mayor General FAP Armando Revoredo Iglesias Ai...,2677
692,MEX,cty-Mexico-28,Licenciado Adolfo Lopez Mateos Intl,2581
